In [9]:
import importlib

import ipynb.fs.full.preprocessing0
import ipynb.fs.full.preprocessing05
#import ipynb.fs.full.prepossessing075_akistage
import ipynb.fs.full.preprocessing1
import ipynb.fs.full.preprocessing2_BT
import ipynb.fs.full.preprocessing25_BTcorr
import ipynb.fs.full.preprocessing3_smote
import ipynb.fs.full.preprocessing4
import ipynb.fs.full.runxgboost
import ipynb.fs.full.postprocessing1_SHAP
import ipynb.fs.full.postprocessing3_collect

from ipynb.fs.full.slackbot import ping_slack
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
#import rpy2.robjects as robjects
#from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
from os.path import exists
import logging
import time

In [10]:
def runner(runner_wrapper, site, year=None, stg=None, fs=None, oversample=None, model_type=None, numberbt=None):
    
    #global setting (no ideal)
    datafolder = '/home/hchan2/AKI/data/'
    home_directory = "/home/hchan2/AKI/AKI_Python/"
    pred_end = 7    
    
#    filename = 'data/'+site+'/model_'+model_type+'_'+site+'_'+str(year)+'_'+stg+'_'+fs+'_'+oversample+'.pkl'
#    if exists(filename):
#        print(filename +" already exists")
#       return

    tic = time.perf_counter() 
    if not numberbt is None:
        runner_wrapper(site, year, stg, fs, oversample, model_type, numberbt)                
    if not fs is None:
        runner_wrapper(site, year, stg, fs, oversample, model_type)        
    elif not stg is None:
        runner_wrapper(site, year, stg)        
    elif not year is None:
        runner_wrapper(site, year)
    else:
        runner_wrapper(site)        
    toc = time.perf_counter()
    print(f"{site}:{year} finished in {toc - tic:0.4f} seconds")        


In [17]:
importlib.reload(ipynb.fs.full.preprocessing0)
importlib.reload(ipynb.fs.full.preprocessing05)
#importlib.reload(ipynb.fs.full.prepossessing075_akistage)
importlib.reload(ipynb.fs.full.preprocessing1)
importlib.reload(ipynb.fs.full.preprocessing2_BT)
importlib.reload(ipynb.fs.full.preprocessing25_BTcorr)
importlib.reload(ipynb.fs.full.preprocessing3_smote)
importlib.reload(ipynb.fs.full.preprocessing4)
importlib.reload(ipynb.fs.full.runxgboost)
importlib.reload(ipynb.fs.full.postprocessing1_SHAP)
importlib.reload(ipynb.fs.full.postprocessing3_collect)

#sites = ['MCRI', 'IUR', 'MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UPITT', 'UTHSCSA', 'KUMC', 'UTSW']
sites = ['MCRI', 'MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UPITT', 'UTHSCSA', 'UTSW']
#sites = ['MCRI', 'UMHC', 'UNMC', 'KUMC']
#sites = ['MCRI']

#sites = ['UPITT']
#sites=['KUMC']
#problematic_sites = ['IUR']

#runner_wrapper_list = [ipynb.fs.full.preprocessing0.read_and_save_lab]

#runner_wrapper_list = [ipynb.fs.full.preprocessing0.read_and_save_onset]
# runner_wrapper_list = [ipynb.fs.full.preprocessing0.read_and_save_demo,
#                        ipynb.fs.full.preprocessing0.read_and_save_vital,
#                        ipynb.fs.full.preprocessing0.read_and_save_dx,
#                        ipynb.fs.full.preprocessing0.read_and_save_px,
#                        ipynb.fs.full.preprocessing0.read_and_save_amed,                      
#                        ipynb.fs.full.preprocessing0.read_and_save_lab]


#runner_wrapper_list = [ipynb.fs.full.preprocessing05.lab_drop_outliner]
#runner_wrapper_list = [ipynb.fs.full.preprocessing05.vital_drop_outliner]

#runner_wrapper_list = [ipynb.fs.full.prepossessing075_akistage.aki_staging]
#runner_wrapper_list = [ipynb.fs.full.prepossessing075_akistage.generate_ckd_table]

#unner_wrapper_list = [ipynb.fs.full.preprocessing1.onset]
# runner_wrapper_list = [ipynb.fs.full.preprocessing1.demo,
#                          ipynb.fs.full.preprocessing1.vital,
#                          ipynb.fs.full.preprocessing1.dx,
#                          ipynb.fs.full.preprocessing1.px,
#                          ipynb.fs.full.preprocessing1.lab,                                              
#                          ipynb.fs.full.preprocessing1.amed]                        

#unner_wrapper_list = [ipynb.fs.full.preprocessing2_BT.bigtable]
#runner_wrapper_list = [ipynb.fs.full.preprocessing25_BTcorr.generate_corr]
#runner_wrapper_list = [ipynb.fs.full.preprocessing25_BTcorr.generate_corr_bt]

#runner_wrapper_list = [ipynb.fs.full.preprocessing4.combinebtpos]
#runner_wrapper_list = [ipynb.fs.full.preprocessing4.combinebt]

#runner_wrapper_list = [ipynb.fs.full.preprocessing3_smote.pre_smote]
#runner_wrapper_list = [ipynb.fs.full.preprocessing3_smote.smote]
#runner_wrapper_list = [ipynb.fs.full.preprocessing3_smote.generate_all_pre_catboost]

#runner_wrapper_list = [ipynb.fs.full.runxgboost.runxgboost]
runner_wrapper_list = [ipynb.fs.full.postprocessing1_SHAP.collectSHAP_sub]

#runner_wrapper_list = [ipynb.fs.full.runxgboost.boosttrapcatboost]

parasites = [(runner, site, None, None, None, None, None, None) for site in sites for runner in runner_wrapper_list]

In [8]:
#year 3000 for all year in one
para_list = []
#stgs = ['stg01', 'stg123']
stgs = ['stg01', 'stg23', 'stg123']
#stgs = ['stg23']
for site in sites:
    years=[3000]
    para_list_local = [(runner, site, year, stg, None, None, None, None) for year in years for stg in stgs for runner in runner_wrapper_list]   
    para_list.extend(para_list_local)

In [46]:
#get years from site
ban_list = [('UPITT', 2013), ('UPITT', 2012), ('MCW', 2011)] #Sample size too small
para_list = []
for site in sites:
    onset = pd.read_pickle('/home/hoyinchan/blue/Data/data2021/data2021/'+site+'/p0_onset_'+site+'.pkl')
    years = list(pd.to_datetime(onset['ADMIT_DATE']).dt.year.unique())    
#    para_list_local = [(runner, site, year, None, None, None, None) for year in years for runner in runner_wrapper_list if not (site, year) in ban_list]   
#    para_list_local = [(runner, site, year, None, None, None, None) for year in years for runner in runner_wrapper_list if (site, year) in ban_list]   
    para_list_local = [(runner, site, year, None, None, None, None, None) for year in years for runner in runner_wrapper_list]   
    para_list.extend(para_list_local)

In [18]:
#for xgboost
#stgs = ["stg123"]
stgs = ["stg23", "stg01"]
fss =  ['nofs', 'rmscrbun']
#stgs = ["stg01"]
#fss =  ['nofs']
#fss =  ['rmlab']
#fss =  ['onlymed']
#oversample = ['raw', 'cp', 'sm']
oversamples = ['raw']
#model_types = ['xgbhgs', 'xgbbs', 'catr']
model_types = ['catd']

#get years from site
ban_list = [('UPITT', 2013), ('UPITT', 2012), ('MCW', 2011)] #Sample size too small
para_listX = []
for site in sites:
#    onset = pd.read_pickle('data/'+site+'/p0_onset_'+site+'.pkl')
#    years = list(pd.to_datetime(onset['ADMIT_DATE']).dt.year.unique())    
    years = [3000]
    para_list_local = [(runner, site, year, stg, fs, oversample, model_type, None) for year in years for stg in stgs for fs in fss for oversample in oversamples for runner in runner_wrapper_list for model_type in model_types if not (site, year) in ban_list]   
    para_listX.extend(para_list_local)
print(len(para_listX))

36


In [ ]:
#for xgboost boosttrap
#stgs = ["stg01"]
stg = ["stg01", "stg23"]
fss =  ['nofs', 'rmscrbun']
#fss =  ['rmscrbun']
fss =  ['nofs']
#oversample = ['raw', 'cp', 'sm']
oversamples = ['raw']
#model_types = ['xgbhgs', 'xgbbs', 'catr']
model_types = ['catd']
number_of_boosttrap = 10

#get years from site
ban_list = [('UPITT', 2013), ('UPITT', 2012), ('MCW', 2011)] #Sample size too small
para_listXX = []
for site in sites:
    for numberbt in range(1,number_of_boosttrap+1):
    #    onset = pd.read_pickle('data/'+site+'/p0_onset_'+site+'.pkl')
    #    years = list(pd.to_datetime(onset['ADMIT_DATE']).dt.year.unique())    
        years = [3000]
        para_list_local = [(runner, site, year, stg, fs, oversample, model_type, numberbt) for year in years for stg in stgs for fs in fss for oversample in oversamples for runner in runner_wrapper_list for model_type in model_types if not (site, year) in ban_list]   
        para_listXX.extend(para_list_local)
print(len(para_listXX))

In [ ]:
import multiprocessing as mp
from pebble import ProcessPool
from concurrent.futures import TimeoutError
def task_done(future):
    try:
        result = future.result()  # blocks until results are ready
    except TimeoutError as error:
        print("Function took longer than %d seconds" % error.args[1])
    except Exception as error:
        print("Function raised " + "error" +"\n"+error.traceback)
        
with ProcessPool(max_workers=20) as pool:
#    for paras in para_listXX:        
    for paras in para_listX:    
#    for paras in para_list:
#    for paras in parasites:
        future = pool.schedule(runner, args=paras, timeout=86400)
        future.add_done_callback(task_done)
        
print('done')
ping_slack('done:AKI_Python:runner.py')

Running shap catd on site MCRI:3000:stg23:nofs:rawRunning shap catd on site MCRI:3000:stg23:rmscrbun:rawRunning shap catd on site MCRI:3000:stg01:nofs:raw

Running shap catd on site MCW:3000:stg23:nofs:raw
Running shap catd on site MCW:3000:stg23:rmscrbun:rawRunning shap catd on site MCW:3000:stg01:nofs:raw/home/hoyinchan/blue/Data/data2021/data2021/MCRI/model_catd_MCRI_3000_stg23_nofs_rawpos.pkl/home/hoyinchan/blue/Data/data2021/data2021/MCRI/model_catd_MCRI_3000_stg23_rmscrbun_rawpos.pkl

Running shap catd on site MCRI:3000:stg01:rmscrbun:raw


/home/hoyinchan/blue/Data/data2021/data2021/MCRI/model_catd_MCRI_3000_stg01_nofs_rawpos.pkl/home/hoyinchan/blue/Data/data2021/data2021/MCW/model_catd_MCW_3000_stg23_nofs_rawpos.pkl/home/hoyinchan/blue/Data/data2021/data2021/MCW/model_catd_MCW_3000_stg23_rmscrbun_rawpos.pkl
/home/hoyinchan/blue/Data/data2021/data2021/MCW/model_catd_MCW_3000_stg01_nofs_rawpos.pkl


/home/hoyinchan/blue/Data/data2021/data2021/MCRI/X_train_MCRI_3000_stg23_rmscrbun